In [1]:
%pylab inline
import pybedtools
import pandas as pd
from pyfaidx import Fasta
import json

cds = pybedtools.BedTool('../data/mm10/annotation/gencode.vM11.gffutils.cds.bed').to_dataframe()#.set_index('name')
gene = pybedtools.BedTool('../data/mm10/annotation/gencode.vM11.gffutils.genes.bed').to_dataframe()#.set_index('name')
utr5 = pybedtools.BedTool('../data/mm10/annotation/gencode.vM11.gffutils.UTR5.bed').to_dataframe()#.set_index('name')
utr3 = pybedtools.BedTool('../data/mm10/annotation/gencode.vM11.gffutils.UTR3.bed').to_dataframe()#.set_index('name')
fasta = Fasta('../data/mm10/fasta/mm10.fa')

def get_fasta_sequence(fasta, intervals):
    if isinstance(fasta, str):
        fasta = Fasta(fasta)
    sequence = []
    for interval in intervals:
        chrom, start, stop, strand = interval
        if strand == '+':
            seq = fasta[chrom][int(start):int(stop)].seq
        elif strand == '-':
            seq = fasta[chrom][int(start):int(stop)].complement.seq
        sequence.append(seq)
    if strand == '-':
        sequence.reverse()# = sequence[::-1]
    return ('').join(sequence).upper()

Populating the interactive namespace from numpy and matplotlib


In [2]:
cds['region_type'] = 'CDS'
gene['region_type'] = 'gene'
utr5['region_type'] = 'UTR5'
utr3['region_type'] = 'UTR3'



In [3]:
master_df = pd.concat([utr5, cds, utr3])

In [ ]:
groups = master_df.groupby('name')

In [ ]:
MASTER_CATEGORIES = ['CDS', 'UTR3', 'UTR5']
def collapse_region(records):
    record = records[0]
    chrom = record['chrom']
    strand = record['strand']
    region = pd.Series(range(record['start'], record['end']))
    for record in records:
        region = region.combine_first(pd.Series(range(record['start'], record['end'])))
    return region

def get_region_index(gene_group):
    
    assert len(gene_group['strand'].unique()) == 1
    assert len(gene_group['chrom'].unique()) == 1
    chrom = gene_group['chrom'].unique()[0]
    strand = gene_group['strand'].unique()[0]

    # Collect all intervals at once
    intervals = zip(gene_group['chrom'], gene_group['start'],
                    gene_group['end'], gene_group['strand'])
    # Need to convert to list instead frm tuples
    # TODO fix this?

    """
    intervals = map(list, intervals)
    if strand == '+':
        # For positive strand shift
        # start codon position intervals[0][1] by -offset
        if intervals[0][1] - master_offset >= 0:
            intervals[0][1] = intervals[0][1] - master_offset
            gene_offset = master_offset
        else:
            sys.stderr.write(
                'Cannot offset beyond 0 for interval: {}. \
                Set to start of chromsome.\n'.format(intervals[0]))
            # Reset offset to minimum possible
            gene_offset = intervals[0][1]
            intervals[0][1] = 0
    else:
        # Else shift cooridnate of last element in intervals stop by + offset
        if (intervals[-1][2] + master_offset <= chrom_length):
            intervals[-1][2] = intervals[-1][2] + master_offset
            gene_offset = master_offset
        else:
            sys.stderr.write('Cannot offset beyond 0 for interval: {}. \
                             Set to end of chromsome.\n'.format(intervals[-1]))
            gene_offset = chrom_length - intervals[-1][2]
            # 1-end so chrom_length
            intervals[-1][2] = chrom_length
    intervals = map(tuple, intervals)

    """

    interval_coverage_list = []
    for index, interval in enumerate(intervals):
        strand = interval[3]
        if strand == '+':
            series_range = range(interval[1], interval[2])
        elif strand == '-':
            series_range = range(interval[2], interval[1], -1)

        series = pd.Series(series_range, index=series_range)
        interval_coverage_list.append(series)


    if len(interval_coverage_list) == 0:
        # Some genes might not be present in the bigwig at all
        sys.stderr.write('Got empty list! intervals  for chr : {}\n'.format(intervals[0][0]))
        return ([], None)

    interval_combined = interval_coverage_list[0]
    for interval_coverage in interval_coverage_list[1:]:
        interval_combined = interval_combined.combine_first(interval_coverage)
    intervals_for_fasta_query = []
    for pos in interval_combined.index:
        if strand == '+':
            intervals_for_fasta_query.append((chrom, pos, pos+1, strand))
        elif strand == '-':
            intervals_for_fasta_query.append((chrom, pos-1, pos, strand))
    return intervals_for_fasta_query


genewise_regions = {}
genewise_lengths = {}
genes_missing_annotation = []
for key, group in groups:
    categories = group.region_type.unique()
    if sorted(categories) !=MASTER_CATEGORIES :
        missing_region = (';').join(list( set(MASTER_CATEGORIES).difference( set(categories) ) ))
        genes_missing_annotation.append((key, missing_region))
        continue
    utr5_rows = group[group.region_type=='UTR5']
    cds_rows = group[group.region_type=='CDS']
    utr3_rows = group[group.region_type=='UTR3']
    
    #utr5_regions = collapse_region(list(utr5_rows.T.to_dict().values()))
    #cds_regions = collapse_region(list(cds_rows.T.to_dict().values()))
    #utr3_regions = collapse_region(list(utr3_rows.T.to_dict().values()))
    utr5_intervals = get_region_index(utr5_rows)
    cds_intervals = get_region_index(cds_rows)
    utr3_intervals = get_region_index(utr3_rows)
    
    utr5_fasta =  get_fasta_sequence(fasta,utr5_intervals)
    cds_fasta =  get_fasta_sequence(fasta,cds_intervals)
    utr3_fasta =  get_fasta_sequence(fasta,utr3_intervals)
    
    genewise_regions[key] = {'UTR5': utr5_fasta,
                             'CDS': cds_fasta,
                             'UTR3' : utr3_fasta}
    genewise_lengths[key] = {'UTR5': len(utr5_fasta),
                             'CDS': len(cds_fasta),
                             'UTR3' : len(utr3_fasta)}
                            


In [ ]:
with open('../data/mm10/input/genes_cds.json', 'w') as outfile:
    json.dump(genewise_regions, outfile)

In [ ]:
with open('../data/mm10/input/genes_lengths.json', 'w') as outfile:
    json.dump(genewise_lengths, outfile)